In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import heapq
import random
import time
import matplotlib.animation as animation

# Define grid size
grid_size = 15

# Generate a grid with obstacles, delivery points, and vehicles
def create_environment():
    grid = np.zeros((grid_size, grid_size))
    
    # Place obstacles
    for _ in range(20):
        x, y = random.randint(0, grid_size-1), random.randint(0, grid_size-1)
        grid[x, y] = 1
    
    # Place vehicles
    vehicles = [(random.randint(0, grid_size-1), random.randint(0, grid_size-1)) for _ in range(10)]
    for x, y in vehicles:
        grid[x, y] = 2
    
    # Place delivery points
    delivery_points = [(random.randint(0, grid_size-1), random.randint(0, grid_size-1)) for _ in range(5)]
    for x, y in delivery_points:
        grid[x, y] = 3
    
    return grid, vehicles, delivery_points

# Plot the environment
def plot_environment(grid, start, goal, path=[]):
    plt.figure(figsize=(10, 10))
    plt.imshow(grid, cmap='Greys', origin='upper')
    
    plt.scatter(start[1], start[0], c='green', s=200, marker='X', label='Start')
    plt.scatter(goal[1], goal[0], c='red', s=200, marker='o', label='Goal')
    
    if path:
        path = np.array(path)
        plt.plot(path[:, 1], path[:, 0], c='blue', linewidth=2, label='Path')
    
    plt.legend()
    plt.show()

# Heuristic function using Euclidean distance
def heuristic(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

# A* Algorithm
def astar(grid, start, goal):
    open_list = []
    heapq.heappush(open_list, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    
    while open_list:
        _, current = heapq.heappop(open_list)
        
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]
        
        neighbors = [(0,1), (1,0), (0,-1), (-1,0)]
        for dx, dy in neighbors:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size and grid[neighbor[0], neighbor[1]] != 1:
                tentative_g_score = g_score[current] + 1
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))
    
    return []

# Best-First Search Algorithm
def best_first_search(grid, start, goal):
    open_list = []
    heapq.heappush(open_list, (heuristic(start, goal), start))
    came_from = {}
    visited = set()
    visited.add(start)
    
    while open_list:
        _, current = heapq.heappop(open_list)
        
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]
        
        neighbors = [(0,1), (1,0), (0,-1), (-1,0)]
        for dx, dy in neighbors:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size and grid[neighbor[0], neighbor[1]] != 1 and neighbor not in visited:
                visited.add(neighbor)
                came_from[neighbor] = current
                heapq.heappush(open_list, (heuristic(neighbor, goal), neighbor))
    
    return []

# Update environment for dynamic changes
def update_environment(grid, delivery_points):
    new_start = delivery_points.pop(0)
    new_goal = random.choice(delivery_points)
    delivery_points.append(new_start)
    return new_start, new_goal

# Path Execution and Control
def execute_path(start, delivery_points):
    current_location = start
    for goal in delivery_points:
        path = astar(grid, current_location, goal)
        if path:
            plot_environment(grid, current_location, goal, path)
            current_location = goal
        new_start, new_goal = update_environment(grid, delivery_points)
        current_location = new_start

# Visualize robot path
def visualize_robot_path(grid, start, goal, path):
    fig, ax = plt.subplots()
    ax.imshow(grid, cmap='Greys', origin='upper')
    
    robot, = ax.plot([], [], 'bo-', markersize=10, label='Robot')
    ax.scatter(start[1], start[0], c='green', s=200, marker='X', label='Start')
    ax.scatter(goal[1], goal[0], c='red', s=200, marker='o', label='Goal')
    
    def init():
        robot.set_data([], [])
        return robot,
    
    def update(frame):
        robot.set_data([p[1] for p in path[:frame+1]], [p[0] for p in path[:frame+1]])
        return robot,
    
    ani = animation.FuncAnimation(fig, update, frames=len(path), init_func=init, blit=True, repeat=False)
    plt.legend()
    plt.show()

# Evaluate performance of algorithms
def evaluate_algorithms(grid, start, goals):
    astar_times = []
    bfs_times = []
    astar_paths = []
    bfs_paths = []
    
    for goal in goals:
        start_time = time.time()
        path = astar(grid, start, goal)
        end_time = time.time()
        astar_times.append(end_time - start_time)
        astar_paths.append(path)
        
        start_time = time.time()
        path = best_first_search(grid, start, goal)
        end_time = time.time()
        bfs_times.append(end_time - start_time)
        bfs_paths.append(path)
    
    return astar_times, bfs_times, astar_paths, bfs_paths

# Main execution
grid, vehicles, delivery_points = create_environment()
start = (0, 0)
goal = delivery_points[0]

# Test the A* algorithm
path = astar(grid, start, goal)
plot_environment(grid, start, goal, path)

# Test the Best-First Search algorithm
path = best_first_search(grid, start, goal)
plot_environment(grid, start, goal, path)

# Test dynamic environment handling
new_start, new_goal = update_environment(grid, delivery_points)
path = astar(grid, new_start, new_goal)
plot_environment(grid, new_start, new_goal, path)

# Test path execution
execute_path(start, delivery_points)

# Test robot path visualization
path = astar(grid, start, goal)
visualize_robot_path(grid, start, goal, path)

# Evaluate algorithms
astar_times, bfs_times, astar_paths, bfs_paths = evaluate_algorithms(grid, start, delivery_points)
print("A* times:", astar_times)
print("BFS times:", bfs_times)
